In [69]:
import sys
import os 
project_root = os.path.abspath("..")
src_path = os.path.join(project_root, "src")

sys.path.insert(0, src_path)
print("Added to path:", src_path)

Added to path: /Users/grace/catenary-wire-modeling/src


In [70]:
import numpy as np
from core import cluster_wires
from models import fit_catenary_wire
from utils import pca_fit_plane
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd


In [79]:
difficulties = ['easy', 'medium', 'hard', 'extrahard']
base_path = "../data/lidar_cable_points_{}.parquet"


cable_points = {
    level: pd.read_parquet(base_path.format(level))
    for level in difficulties
}

In [85]:
points3d = cable_points['extrahard'].values
points3d.shape

(1201, 3)

In [86]:

labels = cluster_wires(points3d)
labels.shape



(1201,)

In [87]:
# Loop over wire clusters and fit catenary 
for wire_id in sorted(set(labels)):
    if wire_id == -1:
        continue  # skip noise

    wire_points = points3d[labels == wire_id]
    result = fit_catenary_wire(wire_points)

    if result is None:
        print(f"   [!] Wire {wire_id}: Fit failed")
        continue

    reconstructed_curve_3d, params = result

    x0, y0, c = params
    print(f"   [✓] Wire {wire_id}: x0={x0:.2f}, y0={y0:.2f}, c={c:.2f}")


   [✓] Wire 0: x0=-0.00, y0=-0.51, c=200.59
   [✓] Wire 1: x0=0.00, y0=-0.51, c=202.18
   [✓] Wire 2: x0=0.37, y0=-0.48, c=201.02


/Users/grace/catenary-wire-modeling/src/models.py:25: OptimizeWarning:

Covariance of the parameters could not be estimated

/Users/grace/catenary-wire-modeling/src/models.py:25: OptimizeWarning:

Covariance of the parameters could not be estimated



In [88]:
# check on the last wire cluster processed
import plotly.express as px
import pandas as pd

df = pd.DataFrame(wire_points, columns=["x", "y", "z"])

fig = px.scatter_3d(df, x="x", y="y", z="z", title="Original 3D Wire Cluster")
fig.update_traces(marker=dict(size=1))  
fig.show()

In [89]:
pts_2d, pca =pca_fit_plane(wire_points)
pts_2d.shape

df = pd.DataFrame(pts_2d, columns=["PC1", "PC2"])


# 2D scatter plot with color per cluster
fig = px.scatter(df, x="PC1", y="PC2", title="PCA Projection by Cluster")
fig.show()